In [1]:
from torch_geometric.utils import scatter

import pathpyG as pp
pp.config['torch']['device'] = 'cpu'

In [2]:
def compute_weighted_outdegrees(graph):
    weighted_outdegree = scatter(graph.data.edge_weight, graph.data.edge_index[0], dim=0, dim_size=graph.data.num_nodes, reduce='sum')
    return weighted_outdegree

def compute_transition_probabilities(graph):
    weighted_outdegree = compute_weighted_outdegrees(graph)
    source_ids = graph.data.edge_index[0]
    return graph.data.edge_weight/ weighted_outdegree[source_ids]


In [3]:
from torch.utils.data import Dataset
import torch

# https://en.wikipedia.org/wiki/Pairing_function
# https://math.stackexchange.com/questions/1377929/generalization-of-cantor-pairing-function-to-triples-and-n-tuples
def cantor_pairing(x, y):
    """
    Computes the Cantor pairing value for two integers x and y.
    The Cantor pairing maps two integers to a unique integer.

    Args:
        x (int): The first integer.
        y (int): The second integer.

    Returns:
        int: The Cantor pairing value for the given integers x and y.
    """
    return (x + y) * (x + y + 1) // 2 + y

def cantor_encode_tensor(tnsr):
    """
    Encodes a list of integers in tensor rows into single integers using Cantor pairing.
    The function recursively applies the Cantor pairing function to pairs of elements
    in the input tensor until it encodes the entire list into a single integer.

    If the input tensor is empty, the function returns 0.

    Args:
        tnsr (torch.Tensor): A tensor containing a list of integers.

    Returns:
        torch.Tensor: The Cantor encoded integer representing the input list.
    """
    if tnsr.size(1) == 0:
        # Termination point. The added 0 has no effect on the returned integer
        return torch.tensor(0)
    else:
        return cantor_pairing(tnsr[:, 0], cantor_encode_tensor(tnsr[:, 1:]))


class WalksDataset(Dataset):
    """
    Dataset class to handle sequences of node walks.

    Args:
        dag_data (object): The input DAG data.
        dict_cantor_to_honode_ixs_mapping (dict): Dictionary mapping Cantor encoded indices to higher-order node indices.
        max_order (int): Maximum order of nodes in a walk.
    """

    def __init__(self, dag_data, dict_cantor_to_honode_ixs_mapping, max_order):
        self.max_order = max_order
        self.dict_cantor_to_honode_ixs_mapping = dict_cantor_to_honode_ixs_mapping
        self._preprocess_data(dag_data.dags)
        if max_order>0:
            self._create_walk_tensors()
            self._create_encoded_tensors()
        else:
            self._create_walk_tensors()
            self.bipartite_encoded_walks_by_length = self.walk_tensors_by_length



    def _preprocess_data(self, dags):
        """
        Preprocesses the DAG data to extract walks and their counts.

        Args:
            dags DAGData object
        """
        self.walks_by_length = {}
        self.walk_counts_by_length = {}
        self.total_sequences = 0
        
        for dag in dags:
            node_seq_path = dag.node_sequence.T[0]
            seq_length = len(node_seq_path)
            if seq_length not in self.walks_by_length:
                self.walks_by_length[seq_length] = []
                self.walk_counts_by_length[seq_length] = []
            self.walks_by_length[seq_length].append(node_seq_path)
            # Answer to Moritz question?
            # probably one weight per DAG would be enough
            self.walk_counts_by_length[seq_length].append(int(dag.edge_weight.unique()))
            self.total_sequences += 1

    def _create_walk_tensors(self):
        """
        Creates tensors for each walk lengths.
        """
        self.walk_tensors_by_length = {
            length: torch.stack(walks, dim=0)
            for length, walks in self.walks_by_length.items()
        }
    def _create_encoded_tensors(self): # TODO: find better names for this method and the _bipartite_encode
        """
        Encode the walk representations using cantor encoding
        """
        self.bipartite_encoded_walks_by_length = {
            length: self._bipartite_encode(length)
            for length in self.walk_tensors_by_length
        }

    def _bipartite_encode(self, walk_length):
        """
        Encodes the walks in bipartite form, i.e.,
        representing transitions between indexes of higher-order nodes.

        In bipartite encoding, each transition in the walk sequence corresponds to a pair of indexes (from nth to n+1th column),
        where the first `self.max_order` transitions utilize indexes from the higher-order nodes of the i-th order tensors, 
        and the subsequent transitions use indexes of the `self.max_order` most recent nodes.

        Args:
            walk_length (int): The length of the walk.

        Returns:
            torch.Tensor: Bipartite-encoded walk sequences.
        """

        list_cantor_node_ixs_tensors = []
        
        for i in range(0, walk_length): 
            hon_ixs_tensor = self.walk_tensors_by_length[walk_length][:, max(0, i - self.max_order):i+1] 
            cantor_encoded = cantor_encode_tensor(hon_ixs_tensor)
            mapped_indices = cantor_encoded.apply_(self.dict_cantor_to_honode_ixs_mapping[min(i+1, self.max_order+1)].get) # +1 cause for first order we want edges etc.
            list_cantor_node_ixs_tensors.append(mapped_indices)
            
        return torch.stack(list_cantor_node_ixs_tensors, dim=1)

    def __getitem__(self, index_tuple):
        """
        Retrieves a bipartite-encoded node sequence from the dataset.

        Args:
            index_tuple (tuple): A tuple containing the length of the sequence (int) and the index of the sequence within that length (int).

        Returns:
            torch.Tensor: The bipartite-encoded node sequence.
        """
        # This cannot work with batching. Could work if each walk length had its own Dataset object
        walk_length, index = index_tuple
        return self.bipartite_encoded_walks_by_length[walk_length][index]

    def __len__(self):
        """
        Returns the total number of sequences in the dataset.

        Returns:
            int: Total number of sequences.
        """
        return self.total_sequences


In [4]:
dag_data = pp.DAGData(pp.IndexMap(list("01234")))

dag_data.append_walk(list("0230230230230"), weight=300)
dag_data.append_walk(list("1241241241241"), weight=300)
dag_data.append_walk(list("0430241"), weight=1)

m = pp.MultiOrderModel.from_DAGs(dag_data, max_order=8)


In [5]:
def get_cantor_econding_mon_nodes(m):
    """ 
    Applies cantor encoding to the node sequences of the higher-order nodes for all the layers of a MultiOrderModel
    Returns a dictionary containing, for each layer, a mapping from the cantor index (encoding) of the sequence to the index of the ho-node in the Graph object
    """
    # zeroth ixs are node ixs
    dict_cantor_to_honode_ixs_mapping = {0:{i:i for i in range(m.layers[1].data.num_nodes)}}
    # This works cause the ho-node sequences are sorted by their indices
    for order, hon in m.layers.items():
        cantor_ids = cantor_encode_tensor(hon.data.node_sequence)
        cantor_to_node_ixs_mapping = {cantor_id.item(): i for i, cantor_id in enumerate(cantor_ids)}
        dict_cantor_to_honode_ixs_mapping[order] = cantor_to_node_ixs_mapping # HEREEEE
    return dict_cantor_to_honode_ixs_mapping
dict_cantor_to_honode_ixs_mapping = get_cantor_econding_mon_nodes(m)

In [6]:
from torch_geometric.loader import DataLoader
def get_mon_likelihood(mon, walk_data, max_order=1):
    """
    Compute the likelihood of the walks given a multi-order model.

    Args:
        mon (MultiOrderModel): The multi-order model.
        walk_data (Dataset): Dataset containing the walks.
        max_order (int, optional): The maximum order up to which model layers
            shall be taken into account. Defaults to 1.

    Returns:
        float: The log likelihood of the walks given the multi-order model.
    """
    assert max_order <= max(mon.layers), "max_order for the walk_data cannot be larger than the max_order in the mon"

    # Get encoding for nodes in the multi-order model
    dict_cantor_to_honode_ixs = get_cantor_econding_mon_nodes(mon)

    # Prepare data for likelihood computation
    inner_walk_data = WalksDataset(walk_data, dict_cantor_to_honode_ixs, max_order=max_order)

    # Compute node emission probabilities for zeroth order
    dag_graph = next(iter(DataLoader(walk_data.dags, batch_size=len(walk_data.dags)))).to(pp.config["torch"]["device"])
    _, counts = torch.unique(dag_graph.node_sequence, return_counts=True)
    node_emission_probabilities = counts / counts.sum()

    # Begin log likelihood computation
    tot_log_lh = 0
    for walk_length, bip_encoded_walks in inner_walk_data.bipartite_encoded_walks_by_length.items():
        path_counts = inner_walk_data.walk_counts_by_length[walk_length]

        # Compute log likelihood for higher orders
        log_lh_path_length = torch.zeros(len(path_counts))
        for i in range(0, walk_length):
            if max_order == 0 or i == 0 :
                T = node_emission_probabilities
            else:
                T = compute_transition_probabilities(m.layers[min(i,max_order)])
                
            target_indexes = bip_encoded_walks[:, i] 
            log_lh_path_length += torch.log(T[target_indexes])
        tot_log_lh += torch.mul(log_lh_path_length, torch.tensor(path_counts)).sum()

    return tot_log_lh.item()

# # Compute likelihood for different maximum orders  
for o in range(max(m.layers) ):
    print(o, get_mon_likelihood(m, dag_data, max_order=o))


0 -12322.8798828125
1 -2658.5830078125
2 -993.5861206054688
3 -993.5485229492188
4 -985.7455444335938
5 -985.7455444335938
6 -985.7455444335938
7 -985.7455444335938


Probably there is somethign wrong in _bipartite_encode

In [7]:
def get_mon_dof(m, max_order=None, assumption="paths"):
    """
    The degrees of freedom fo the kth layer of a multi-order model this depende on the number of different paths of exactly length k in the graph.
    Therefore, we can obtain this values by summing the entries of the kth power of the binary adhacency matrix of the graph.
    Finally, we must consider that, due the conservation of probablility, all non-zero rows of the transition matrix of the higher-order network must sum to one. 
    This poses on additional constraint per row that respects the condition, which should be removed from the total count of degrees of freedom.

    Args:
        m (MultiOrderModel): The multi-order model.
        max_order (int, optional): The maximum order up to which model layers 
            shall be taken into account. Defaults to None, meaning it considers 
            all available layers.
        assumption (str, optional): If set to 'paths', only paths in the 
            first-order network topology will be considered for the degree of 
            freedom calculation. If set to 'ngrams', all possible n-grams will 
            be considered, independent of whether they are valid paths in the 
            first-order network or not. Defaults to 'paths'.

    Returns:
        int: The degrees of freedom for the multi-order model.

    Raises:
        AssertionError: If max_order is larger than the maximum order of 
            the multi-order network.
        ValueError: If the assumption is not 'paths' or 'ngrams'.
    """
    if max_order is None:
        max_order = max(m.layers)
    
    assert max_order <= max(m.layers), "Error: max_order cannot be larger than maximum order of multi-order network"

    dof = m.layers[1].data.num_nodes - 1  # Degrees of freedom for zeroth order

    if assumption == "paths":
        # COMPUTING CONTRIBUTION FROM NUM PATHS AND NONERO OUTDEGREES SEPARATELY
        # TODO: CAN IT BE DONE TOGETHER?

        # Adding dof from Number of paths of length k 
        for k in range(1, max_order + 1):
            
            if k == 1:
                edge_index = m.layers[1].data.edge_index
            else:
                edge_index = m.lift_order_edge_index(edge_index, num_len_k_paths)
            num_len_k_paths = edge_index.shape[1]  # Number of paths of length k
            dof += num_len_k_paths 
        
        # removing dof from total probability of nonzero degree nodes
        for k in range(1, max_order+1):
            
            if k == 1:
                edge_index_adj = m.layers[1].data.edge_index
                edge_index = edge_index_adj
            else:
                edge_index, _ = edge_index @ edge_index_adj
            num_nonzero_outdegrees = torch.unique(edge_index[0]).size(0)
            dof -=  num_nonzero_outdegrees

      
    elif assumption == "ngrams":
        for order in range(1, max_order + 1):
            dof += (m.layers[1].data.num_nodes ** order) * (m.layers[1].data.num_nodes - 1)
    else:
        raise ValueError(f"Unknown assumption {assumption} in input. The only accepted values are 'path' and 'ngram'")

    return int(dof)

for o in range(0,6):
    # get_mon_dof(m,max_order=o)
    print("dof",o,"-",get_mon_dof(m,max_order=o))

dof 0 - 4
dof 1 - 7
dof 2 - 15
dof 3 - 30
dof 4 - 57
dof 5 - 104


/opt/conda/lib/python3.10/site-packages/torch_geometric/edge_index.py:784: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(


In [8]:
from scipy.stats import chi2
def lh_ratio_test(mon, walk_data, max_order_null = 0, max_order = 1, assumption='paths', significance_threshold=0.01):
    assert max_order_null < max_order, 'Error: order of null hypothesis must be smaller than order of alternative hypothesis'
    assert max_order < max(mon.layers), f'Error: order of hypotheses ({max_order_null} and {max_order}) must be smaller than the maximum order of the MultiOrderModel {max(mon.layers)}'
    # let L0 be the likelihood for the null model and L1 be the likelihood for the alternative model

    # we first compute a test statistic x = -2 * log (L0/L1) = -2 * (log L0 - log L1)
    # get_mon_likelihood(mon, walk_data)
    x = -2 * (get_mon_likelihood(mon, walk_data, max_order=max_order_null) - get_mon_likelihood(mon, walk_data, max_order=max_order))

    # we calculate the additional degrees of freedom in the alternative model
    dof_diff = get_mon_dof(m,max_order, assumption = assumption) - get_mon_dof(m,max_order_null, assumption = assumption)
    print(x, dof_diff)

    # if the p-value is *below* the significance threshold, we reject the null hypothesis
    p = 1-chi2.cdf(x, dof_diff)
    return (p<significance_threshold), p

Mess originates from the fact that mon and walk dataset can have incompatbile values. 

In [9]:
lh_ratio_test(m, dag_data, max_order_null = 1, max_order = 2, assumption='paths', significance_threshold=0.01)

3329.9937744140625 8


(True, 0.0)

In [10]:
def estimate_order(mon, walk_data, max_order=None, significance_threshold=0.01):
    """
    Selects the optimal maximum order of a multi-order network model for the
    observed paths, based on a likelihood ratio test with p-value threshold of p
    By default, all orders up to the maximum order of the multi-order model will be tested.

    @param paths: The path statistics for which to perform the order selection

    @param maxOrder: The maximum order up to which the multi-order model shall be tested.
    """
    if max_order == None:
        max_order = mon.max_order
    assert max_order <= max(mon.layers), 'Error: maxOrder cannot be larger than maximum order of multi-order network'
    assert max_order > 1, 'Error: maxOrder must be larger than one'

    max_accepted_order = 1

    # Test for highest order that passes
    # likelihood ratio test against null model
    for k in range(2, max_order+1):
        if lh_ratio_test(m, walk_data, max_order_null = k-1, max_order = k, significance_threshold=significance_threshold)[0]:
            max_accepted_order = k

    return max_accepted_order

estimate_order(m, dag_data, max_order=max(m.layers)-1) # this -1 indicates that something is still off

3329.9937744140625 8


0.0751953125 15
15.60595703125 27
-0.0 47
-0.0 76
-0.0 124


2